In [1]:
%load_ext rpy2.ipython

In [2]:
%%R
knitr::opts_chunk$set(echo = TRUE)

nInd<-175
indIDs <-paste0("X", sample(10000:40000, nInd))
nVisits<-rpois(indIDs, 4)
nVisits[which(nVisits == 0)]<-1

cogAbaseline<-rpois(indIDs, 25)
cogBbaseline<-rnorm(indIDs, 8,4)
cogCbaseline<-rnorm(indIDs, 20, 2)
sex<-sample(c("M", "F"), nInd, replace = TRUE, prob = c(0.55, 0.45))
age<-floor(runif(nInd, 20, 60))
intervention<-sample(c("Placebo", "Training"), nInd, replace = TRUE)
yearsEd<-sample(c(12,14,17), nInd, replace = TRUE, prob = c(0.3,0.4, 0.3))
smoke <- sample(c("Yes", "No"), nInd, replace = TRUE, prob = c(0.25,0.75))
physicalWellbeing <- sample(c("High", "Low"), nInd, replace = TRUE, prob = c(0.85,0.15))
mentalWellbeing <- sample(c("High", "Low"), nInd, replace = TRUE, prob = c(0.7,0.3))
cogAbaseline <- cogAbaseline[(physicalWellbeing == "Low" | mentalWellbeing == "Low")]<- rpois(sum((physicalWellbeing == "Low" | mentalWellbeing == "Low")), 22)
cogAbaseline <- cogAbaseline[smoke == "Yes"]<- rpois(sum(smoke == "Yes"), 23)

visitID<-as.factor(rep(indIDs, nVisits))
visitNum <- unlist(lapply(nVisits, seq))

index<-match(visitID, indIDs)
visitSex<-as.factor(sex[index])
visitAge<-age[index]+visitNum
visitIntervention<-as.factor(intervention[index])
visitYearsEd <- yearsEd[index]
visitSmoke <- as.factor(smoke[index])
visitPW <- as.factor(physicalWellbeing[index])
randomIndex<-sample(1:length(index), nInd)
visitPW[randomIndex]<-"Low"
visitMW <-as.factor(mentalWellbeing[index])
randomIndex<-sample(which(visitNum > 3), nInd*0.5)
visitMW[randomIndex]<-"High"
    
cogA<- floor(cogAbaseline[index] + visitNum * (0.2 + 0.05 * as.numeric(visitIntervention) + 0.04 * as.numeric(visitMW)) + rnorm(length(visitNum), 0,2))

cogB<-cogBbaseline[index] + visitNum * (0.1 - 0.08 * as.numeric(visitSex) + 0.05 * (visitYearsEd-12)) + rnorm(length(visitNum), 0, 1) 

cogC<-cogCbaseline[index] + visitNum * (0.01 + 0.003 * as.numeric(visitSex) + 0.001 * as.numeric(visitIntervention)) + rnorm(length(visitNum), 0, 5) 


cogDat<-data.frame("ID" = visitID, "VisitNum" = visitNum, "Age" = visitAge, "Sex" = visitSex, "YearsEducation" = visitYearsEd, "Smoker" = visitSmoke, "Intervention" = visitIntervention, "CognitionA" = cogA, "CognitionB" = cogB, "CognitionC" = cogC, "PhysicalWellbeing" = visitPW, "MentalWellbeing" = visitMW)


# Extras

## R coding conventions for interactions

In fact we can write this code more compactly, as R will automatically include the main effects for the two variables as well as the interaction, if we use the `*` to denote which variables we want to model an interaction for. For example, we obtain the same output with the more compact coding here:

In [3]:
%%R
model.int<-lm(CognitionB ~ VisitNum*Sex, dat = cogDat)
summary(model.int)


Call:
lm(formula = CognitionB ~ VisitNum * Sex, data = cogDat)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.0439  -2.9890  -0.2351   2.9613  11.4649 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)     8.4569     0.4566  18.523   <2e-16 ***
VisitNum        0.2655     0.1329   1.997   0.0461 *  
SexM           -0.4223     0.6104  -0.692   0.4893    
VisitNum:SexM  -0.3016     0.1790  -1.685   0.0924 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.095 on 720 degrees of freedom
Multiple R-squared:  0.02987,	Adjusted R-squared:  0.02583 
F-statistic:  7.39 on 3 and 720 DF,  p-value: 7.003e-05



If in fact, we want to include just the interaction without the main effect terms, we can use ":" instead.

For example:

In [4]:
%%R
model.int<-lm(CognitionB ~ VisitNum:Sex, dat = cogDat)
summary(model.int)


Call:
lm(formula = CognitionB ~ VisitNum:Sex, data = cogDat)

Residuals:
    Min      1Q  Median      3Q     Max 
-9.9809 -2.9682 -0.2984  2.9523 11.4965 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)    8.22064    0.30295  27.136  < 2e-16 ***
VisitNum:SexF  0.32450    0.10189   3.185  0.00151 ** 
VisitNum:SexM -0.08393    0.09782  -0.858  0.39118    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.093 on 721 degrees of freedom
Multiple R-squared:  0.02923,	Adjusted R-squared:  0.02654 
F-statistic: 10.85 on 2 and 721 DF,  p-value: 2.268e-05



Because we have omitted the main effect terms, we need two interaction terms to capture the sex specific effects (i.e. we need two regression coefficients to enable us to estimate a female-specific slope and a male-specific slope). When we have age as a main effect, the regression coefficient is equivalent to the "VisitNum:SexF" variable. As shown here


In [5]:
%%R
model.int<-lm(CognitionB ~ Sex + VisitNum:Sex, dat = cogDat)
summary(model.int)


Call:
lm(formula = CognitionB ~ Sex + VisitNum:Sex, data = cogDat)

Residuals:
     Min       1Q   Median       3Q      Max 
-10.0439  -2.9890  -0.2351   2.9613  11.4649 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)    8.45687    0.45656  18.523   <2e-16 ***
SexM          -0.42230    0.61044  -0.692   0.4893    
SexF:VisitNum  0.26549    0.13291   1.997   0.0461 *  
SexM:VisitNum -0.03607    0.11984  -0.301   0.7635    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 4.095 on 720 degrees of freedom
Multiple R-squared:  0.02987,	Adjusted R-squared:  0.02583 
F-statistic:  7.39 on 3 and 720 DF,  p-value: 7.003e-05



In general though, it is advisable to have the main effects for each predictor variable as well as the interaction, to ensure that effects are correctly attributed to the right source.

## More complex mixed effects models

Take a look at the [lme4 vignette](https://cran.r-project.org/web/packages/lme4/vignettes/lmer.pdf) for more details on how to specify more complex mixed effect models with this package.

Also this post: <https://rstudio-pubs-static.s3.amazonaws.com/63556_e35cc7e2dfb54a5bb551f3fa4b3ec4ae.html>

Notes on REML here: <http://users.stat.umn.edu/~gary/classes/5303/handouts/REML.pdf>

A common error message when using `lmer()` is `Error in KhatriRao(sm, t(mm)) : (p \<- ncol(X)) == ncol(Y) is not TRUE`

If you get this error, try removing observations with missing data. While `lm()` and `glm()` were good at automatically handling the presence of these lmer throws an arguably confusing error.